In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Conv2DTranspose, LeakyReLU, Flatten, Reshape, BatchNormalization, Input, UpSampling2D, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os
import glob
from google.colab import drive
from IPython.display import display, HTML


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (Dense, Conv2D, UpSampling2D, LeakyReLU, Flatten, Reshape, BatchNormalization, Input, Activation)
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from google.colab import drive
import numpy as np
import os
import glob
from IPython.display import display, HTML

# ---------------------------
# Mount Google Drive
# ---------------------------
drive.mount('/content/drive')

# ---------------------------
# Dataset Loader
# ---------------------------
def load_dataset(image_folder, target_size=(224, 224)):
    image_paths = glob.glob(os.path.join(image_folder, "*.jpg"))
    images = []
    for img_path in image_paths:
        img = load_img(img_path, target_size=target_size)
        img = img_to_array(img) / 127.5 - 1.0  # Normalize to [-1, 1]
        images.append(img)
    return np.array(images)

dataset_path = '/content/drive/MyDrive/House_Data_2'
dataset = load_dataset(dataset_path)
print(f"Loaded dataset with {dataset.shape[0]} images.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded dataset with 3695 images.


In [ ]:
from tensorflow.keras.layers import Lambda

def build_generator(latent_dim):
    latent_input = Input(shape=(latent_dim,))

    # Mapping Network
    x = Dense(128)(latent_input)
    for _ in range(7):
        x = LeakyReLU(alpha=0.2)(x)
        x = Dense(128)(x)

    # Start with low-resolution (4x4)
    x = Dense(4 * 4 * 512)(latent_input)
    x = Reshape((4, 4, 512))(x)
    x = LeakyReLU(alpha=0.2)(x)

    # Progressive Growing
    filters = [512, 256, 128, 64]
    resolutions = [8, 16, 32, 64]  # Correct target resolutions
    for f, res in zip(filters, resolutions):
        x = UpSampling2D()(x)  # Double resolution
        x = Conv2D(f, kernel_size=3, padding="same")(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.2)(x)
        print(f"Current resolution: {res}x{res}")

    # Final Adjustment to 224x224
    # Scale from 64x64 to 224x224
    x = UpSampling2D(size=(3, 3))(x)  # Scale directly to 192x192
    x = Conv2D(3, kernel_size=3, padding="same", activation="tanh")(x)

    # Final 224x224 adjustment using Lambda layer
    x = Lambda(lambda img: tf.image.resize(img, size=(224, 224)))(x)

    model = Model(latent_input, x, name="Generator")
    return model


In [ ]:


# ---------------------------
# StyleGAN-Inspired Generator
# ---------------------------
# ---------------------------
# Deep CNN-Based Discriminator
# ---------------------------
def build_discriminator(image_shape):
    img_input = Input(shape=image_shape)

    # Downsampling blocks
    filters = [64, 128, 256, 512, 1024]
    x = img_input
    for f in filters:
        x = Conv2D(f, kernel_size=4, strides=2, padding="same")(x)
        x = LeakyReLU(alpha=0.2)(x)
        x = BatchNormalization()(x)

    # Final classification
    x = Flatten()(x)
    validity = Dense(1, activation="sigmoid")(x)
    model = Model(img_input, validity, name="Discriminator")
    return model

# ---------------------------
# GAN Combined Model
# ---------------------------
def build_gan(generator, discriminator):
    discriminator.trainable = False
    latent_input = Input(shape=(latent_dim,))
    generated_img = generator(latent_input)
    validity = discriminator(generated_img)
    model = Model(latent_input, validity, name="GAN")
    return model

# ---------------------------
# Training Functionality
# ---------------------------
stop_training_flag = False

def save_models(generator, discriminator, epoch, path="/content/drive/MyDrive/GAN_Checkpoints"):
    os.makedirs(path, exist_ok=True)
    generator.save(os.path.join(path, f"generator_epoch_{epoch}.h5"))
    discriminator.save(os.path.join(path, f"discriminator_epoch_{epoch}.h5"))
    generator.save_weights(os.path.join(path, f"generator_weights_epoch_{epoch}.h5"))
    discriminator.save_weights(os.path.join(path, f"discriminator_weights_epoch_{epoch}.h5"))

def generate_and_save_images(generator, epoch, latent_dim, save_dir="/content/drive/MyDrive/GAN_Results"):
    os.makedirs(save_dir, exist_ok=True)
    noise = np.random.normal(0, 1, (16, latent_dim))
    generated_images = generator.predict(noise)
    generated_images = (generated_images + 1) * 127.5  # Rescale to [0, 255]
    generated_images = generated_images.astype(np.uint8)

    import matplotlib.pyplot as plt
    fig, axes = plt.subplots(4, 4, figsize=(8, 8))
    for i, ax in enumerate(axes.flat):
        ax.imshow(generated_images[i])
        ax.axis('off')
    plt.savefig(os.path.join(save_dir, f"epoch_{epoch}.png"))
    plt.close()

def stop_training():
    global stop_training_flag
    stop_training_flag = True
    print("Training will stop after this epoch.")

def train_gan(generator, discriminator, gan, dataset, epochs, batch_size, latent_dim):
    global stop_training_flag
    for epoch in range(1, epochs + 1):
        if stop_training_flag:
            break

        # Train Discriminator
        for _ in range(batch_size):
            idx = np.random.randint(0, dataset.shape[0], batch_size)
            real_imgs = dataset[idx]
            real_labels = np.ones((batch_size, 1))

            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            fake_imgs = generator.predict(noise)
            fake_labels = np.zeros((batch_size, 1))

            d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)
            d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
            d_loss = [
                0.5 * (d_loss_real[0] + d_loss_fake[0]),  # Average loss
                0.5 * (d_loss_real[1] + d_loss_fake[1])   # Average accuracy
            ]

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_labels)

        # Extract g_loss value if it is a tensor or list
        if isinstance(g_loss, (list, tuple)):
            g_loss = g_loss[0]  # Take the first element if it’s a list/tuple
        elif hasattr(g_loss, "numpy"):
            g_loss = g_loss.numpy()  # Convert to NumPy if it’s a tensor

        # Log progress
        print(f"Epoch {epoch}/{epochs} [D loss: {d_loss[0]:.4f} | D accuracy: {100 * d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")

        # Save generated images and models at intervals
        if epoch % 100 == 0 or epoch == epochs:
            generate_and_save_images(generator, epoch, latent_dim)
            save_models(generator, discriminator, epoch)

    if stop_training_flag:
        print("Training stopped by user.")
    save_models(generator, discriminator, "final")



# ---------------------------
# Add Stop Button
# ---------------------------
display(HTML("""
<button onclick="google.colab.kernel.invokeFunction('stop_training', [], {})">Stop Training</button>
"""))

# ---------------------------
# Parameters and Start Training
# ---------------------------
latent_dim = 128
image_shape = (224, 224, 3)
epochs = 5000
batch_size = 32

generator = build_generator(latent_dim)
discriminator = build_discriminator(image_shape)
gan = build_gan(generator, discriminator)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.5, beta_2=0.999)
discriminator.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
gan.compile(optimizer=optimizer, loss="binary_crossentropy")

train_gan(generator, discriminator, gan, dataset, epochs, batch_size, latent_dim)


Current resolution: 8x8
Current resolution: 16x16
Current resolution: 32x32
Current resolution: 64x64
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
